In [ ]:
# Ensure GPU runtime
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets sentencepiece # Dependencies
print("=== Installation Complete ===")

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-unozflah/unsloth_8b7d339dd0c547d0b01e5f89ef8a3894
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-unozflah/unsloth_8b7d339dd0c547d0b01e5f89ef8a3894
  Resolved https://github.com/unslothai/unsloth.git to commit c9b9a366e7a6110f9d58d5ed8db6bd27bc97fb71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 10.5 MB/s eta 0:00:0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2
=== Installation Complete ===


In [ ]:
import torch
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
import os
import gc
import time
# from huggingface_hub import login # Login might be needed for Phi-3

# Optional Login:
# try:
#     login("hf_YOUR_HUGGINGFACE_TOKEN")
#     print("HF Login successful.")
# except Exception as e:
#     print(f"HF Login skipped/failed: {e}")

print("=== Imports Complete ===")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
=== Imports Complete ===


In [ ]:
# --- Model & Training Config ---
# Using Phi-3 Mini Instruct as suggested by hint article
model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 2048 # Phi-3 Mini supports up to 4k, use 2k for balance
dtype = None # Auto-detect
load_in_4bit = True

# --- Dataset Config ---
# Dataset suggested by hint article
dataset_name = "Amod/mental_health_counseling_conversations"
dataset_subset_size = 1000 # Use a subset for faster assignment run
# Columns expected (verify in Cell 6): Context, Response

# --- Output & Training Params ---
output_directory = "phi3_mini_mental_health_chatbot_run1"
# Use fewer steps for assignment demo
training_max_steps = 75 # Adjust as needed for speed/performance balance
learning_rate = 2e-4 # Standard SFT LR

print(f"--- Mental Health Chatbot Configuration ---")
print(f"  Model: {model_name}")
print(f"  Max Seq Length: {max_seq_length}")
print(f"  Dataset: {dataset_name}")
print(f"  Subset Size: {dataset_subset_size}")
print(f"  Max Steps: {training_max_steps}")
print(f"  Learning Rate: {learning_rate}")
print(f"  Output Dir: {output_directory}")
print("=== Configuration Set ===")

--- Mental Health Chatbot Configuration ---
  Model: unsloth/Phi-3-mini-4k-instruct-bnb-4bit
  Max Seq Length: 2048
  Dataset: Amod/mental_health_counseling_conversations
  Subset Size: 1000
  Max Steps: 75
  Learning Rate: 0.0002
  Output Dir: phi3_mini_mental_health_chatbot_run1
=== Configuration Set ===


In [ ]:
print("--- Loading Model & Tokenizer ---")
start_time = time.time()
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
except Exception as e: print(f"Error loading model: {e}"); raise
end_time = time.time()
print(f"Model loaded in {end_time - start_time:.2f}s.")

# *** IMPORTANT: Set Chat Template if not automatically detected ***
# Phi-3 uses a specific ChatML-like template.
if tokenizer.chat_template is None:
    print("WARNING: tokenizer.chat_template is None. Manually setting Phi-3 template.")
    # Reference: https://huggingface.co/microsoft/Phi-3-mini-4k-instruct#chat-format
    phi3_template = """{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|' + message['role'] + '|>\n' + message['content'] | trim + '<|end|>\n' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% endif %}"""
    tokenizer.chat_template = phi3_template
    print("Manually set Phi-3 chat template.")
else:
    print("tokenizer.chat_template is already set.")

# Ensure special tokens are set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Set pad_token = eos_token.")
if tokenizer.bos_token is None: print("Warning: BOS token not set in tokenizer.") # Phi-3 template uses bos_token
if tokenizer.eos_token is None: print("Warning: EOS token not set in tokenizer.") # Critical

print("=== Model and Tokenizer Loaded ===")

--- Loading Model & Tokenizer ---
==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.50.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Model loaded in 22.25s.
tokenizer.chat_template is already set.
=== Model and Tokenizer Loaded ===


In [ ]:
print("--- Configuring LoRA ---")
try:
    model = FastLanguageModel.get_peft_model(
        model,
        r = 16, # Rank 16 is often a good starting point
        lora_alpha = 32,
        lora_dropout = 0.05, # Add slight dropout for regularization
        bias = "none",
        use_gradient_checkpointing = True,
        random_state = 3407,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj",],
    )
    print("LoRA configured:")
    print(model.print_trainable_parameters())
except Exception as e: print(f"Error configuring LoRA: {e}"); raise
print("=== LoRA Configured ===")

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


--- Configuring LoRA ---


Unsloth 2025.3.19 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


LoRA configured:
trainable params: 29,884,416 || all params: 3,850,963,968 || trainable%: 0.7760
None
=== LoRA Configured ===


In [ ]:
print(f"--- Loading Dataset: {dataset_name} ---")
try:
    dataset = load_dataset(dataset_name, split="train") # Assuming data is in train split
    print(f"Dataset loaded (initial size: {len(dataset)}).")

    # Select a subset for faster processing
    if dataset_subset_size < len(dataset):
         dataset = dataset.shuffle(seed=42).select(range(dataset_subset_size))
    print(f"Selected subset of {len(dataset)} examples.")

    # *** CRITICAL: Inspect features to confirm column names ***
    print("\nDataset features:", dataset.features)
    # Expected columns based on dataset card/hint article: 'Context', 'Response'
    # Verify these exist.
    required_columns = ['Context', 'Response']
    missing_cols = [col for col in required_columns if col not in dataset.features]
    if missing_cols:
        print(f"\n*** ERROR: Required columns {missing_cols} not found in dataset! Features found: {list(dataset.features.keys())} ***")
        raise KeyError(f"Missing required dataset columns: {missing_cols}")

    print("\nFirst example:")
    print(dataset[0])

except Exception as e:
    print(f"Error loading or processing dataset '{dataset_name}': {e}")
    raise

print("=== Dataset Loaded and Inspected ===")

--- Loading Dataset: Amod/mental_health_counseling_conversations ---


README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Dataset loaded (initial size: 3512).
Selected subset of 1000 examples.

Dataset features: {'Context': Value(dtype='string', id=None), 'Response': Value(dtype='string', id=None)}

First example:
{'Context': "I didn't trust my wife when I found out that she had a new guy friend that she was texting and calling. I investigated him before I found out that he was gay and that there was nothing going on. Now all my wife and I do is fight about trust.", 'Response': 'Instead of fighting about trust, is it possible for you and your wife to talk with other about areas which upset each of you?Whenever feelings are hurt, knowing what exactly is problematic and being heard and understood by the partner, goes a long way to building trust.These type of discussions are hard to have, and especially for the first time. \xa0A lot of emotions arise and often people lose their conversation focus from this.If you and your wife have a tough time opening up to each other, consider scheduling time with a coupl

In [ ]:
print("--- Defining Formatting Function ---")

# *** THIS IS THE MOST IMPORTANT PART FOR SAFETY ***
# Define the system prompt with clear disclaimers.
# Modify the wording as needed, but keep the core message.
SYSTEM_PROMPT = """You are 'MindMate', a supportive AI assistant trained to provide empathetic listening and helpful information related to mental well-being based on the user's input.

**Important Limitations:**
*   I am an AI and cannot provide medical advice, diagnosis, or treatment.
*   I am not a substitute for a qualified mental health professional (like a therapist or counselor).
*   My purpose is to offer general support, information, and a safe space to talk.
*   **If you are in crisis or feel you might harm yourself or others, please contact a crisis hotline, emergency services (like 911 or your local equivalent), or seek help from a professional immediately.**

Please share what's on your mind, and I'll do my best to listen and respond supportively within my capabilities."""

print(f"Using System Prompt:\n------\n{SYSTEM_PROMPT}\n------")

# Define formatting function using dataset columns identified in Cell 6
# Assumes columns are 'Context' (user input) and 'Response' (assistant output)
def format_mental_health_prompt(example):
    user_context = example.get("Context", "") # Get user input safely
    assistant_response = example.get("Response", "") # Get target response safely

    # Structure according to the Phi-3 Chat Template
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_context},
        {"role": "assistant", "content": assistant_response}
    ]

    # Apply the template (already set on the tokenizer in Cell 4)
    try:
        # add_generation_prompt=False because we provide the full turn for training
        formatted_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        return {"text": formatted_text} # Output column named 'text'
    except Exception as e:
        print(f"Error applying template to example: {e}")
        # print(f"Example data: Context='{user_context}', Response='{assistant_response}'")
        return {"text": ""} # Return empty on error

print("Formatting function `format_mental_health_prompt` defined.")
print("=== Formatting Function Defined ===")

--- Defining Formatting Function ---
Using System Prompt:
------
You are 'MindMate', a supportive AI assistant trained to provide empathetic listening and helpful information related to mental well-being based on the user's input.

**Important Limitations:**
*   I am an AI and cannot provide medical advice, diagnosis, or treatment.
*   I am not a substitute for a qualified mental health professional (like a therapist or counselor).
*   My purpose is to offer general support, information, and a safe space to talk.
*   **If you are in crisis or feel you might harm yourself or others, please contact a crisis hotline, emergency services (like 911 or your local equivalent), or seek help from a professional immediately.**

Please share what's on your mind, and I'll do my best to listen and respond supportively within my capabilities.
------
Formatting function `format_mental_health_prompt` defined.
=== Formatting Function Defined ===


In [ ]:
print("--- Applying Formatting to Dataset ---")
# Make sure dataset object exists from Cell 6
if 'dataset' not in locals(): raise NameError("Dataset not loaded. Run Cell 6 first.")

try:
    original_cols = list(dataset.features) # Get columns to remove
    dataset = dataset.map(
        format_mental_health_prompt,
        # batched=True, # Batched=True might be faster but can hide individual errors
        num_proc=2, # Use multiprocessing
        remove_columns=original_cols # Remove original 'Context', 'Response'
    )
    print("Formatting applied.")
    # Filter out empty examples resulting from formatting errors
    initial_len = len(dataset)
    dataset = dataset.filter(lambda x: len(x['text']) > 0)
    final_len = len(dataset)
    if initial_len != final_len: print(f"Filtered out {initial_len - final_len} empty examples.")

    print("Processed dataset features:", dataset.features)
    if final_len > 0:
        print("\nExample formatted text (first 500 chars):")
        print(dataset[0]['text'][:500])
    else: print("\nWarning: Dataset is empty after formatting/filtering.")

except Exception as e: print(f"Error mapping dataset: {e}"); raise

print("=== Formatting Applied ===")

--- Applying Formatting to Dataset ---


Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Formatting applied.


Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Processed dataset features: {'text': Value(dtype='string', id=None)}

Example formatted text (first 500 chars):
<|system|>
You are 'MindMate', a supportive AI assistant trained to provide empathetic listening and helpful information related to mental well-being based on the user's input.

**Important Limitations:**
*   I am an AI and cannot provide medical advice, diagnosis, or treatment.
*   I am not a substitute for a qualified mental health professional (like a therapist or counselor).
*   My purpose is to offer general support, information, and a safe space to talk.
*   **If you are in crisis or feel 
=== Formatting Applied ===


In [ ]:
print("--- Configuring Trainer ---")
if 'model' not in locals() or 'tokenizer' not in locals() or 'dataset' not in locals():
    raise NameError("Model, tokenizer, or dataset not defined. Run previous cells.")
if len(dataset) == 0: raise ValueError("Cannot configure trainer with empty dataset.")

try:
    trainer = SFTTrainer(
        model=model, # LoRA enabled model
        tokenizer=tokenizer,
        train_dataset=dataset,
        dataset_text_field="text", # Column containing formatted text
        max_seq_length=max_seq_length, # From Cell 3
        dataset_num_proc=2,
        packing=True, # Packing is good for conversational data

        args=TrainingArguments(
            per_device_train_batch_size=2, # Adjust based on VRAM
            gradient_accumulation_steps=8, # Effective batch size 16
            warmup_steps=10,
            max_steps=training_max_steps, # Use max steps from Cell 3
            learning_rate=learning_rate, # Use LR from Cell 3
            fp16=not torch.cuda.is_bf16_supported(),
            bf16=torch.cuda.is_bf16_supported(),
            logging_steps=5, # Log frequently for short run
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=3407,
            output_dir=output_directory, # Output dir from Cell 3
            save_strategy="steps",
            save_steps= max(1, training_max_steps // 2), # Save roughly midway
            report_to="none", # Disable external reporting
        ),
    )
    print("Trainer configured.")
except Exception as e: print(f"Error configuring trainer: {e}"); raise
print("=== Trainer Configured ===")

--- Configuring Trainer ---


Generating train split: 0 examples [00:00, ? examples/s]

Trainer configured.
=== Trainer Configured ===


In [ ]:
print(f"--- Starting Training (max_steps={training_max_steps}) ---")
gc.collect(); torch.cuda.empty_cache(); print("Cleared CUDA cache.")
start_train_time = time.time()
try:
    trainer.train()
    end_train_time = time.time()
    print(f"\nTraining finished in {(end_train_time - start_train_time)/60:.2f} minutes.")
except Exception as e: print(f"\n--- ERROR DURING TRAINING ---"); print(e); import traceback; traceback.print_exc(); print("-----------------------------")
print("=== Training Complete ===")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 244 | Num Epochs = 5 | Total steps = 75
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 29,884,416/4,000,000,000 (0.75% trained)


--- Starting Training (max_steps=75) ---
Cleared CUDA cache.
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,1.994500
10,1.715200
15,1.547100
20,1.455600
25,1.350500
30,1.381900
35,1.329400
40,1.327800
45,1.316200
50,1.305300



Training finished in 31.34 minutes.
=== Training Complete ===


In [ ]:
final_adapter_dir = f"{output_directory}/final_adapters"
os.makedirs(final_adapter_dir, exist_ok=True)
print(f"\n--- Saving final LoRA adapters to: {final_adapter_dir} ---")
try:
    trainer.model.save_pretrained(final_adapter_dir) # Save PEFT model
    tokenizer.save_pretrained(final_adapter_dir) # Save tokenizer
    print("Adapters and tokenizer saved.")
    !ls -lh {final_adapter_dir} # Verify files
except Exception as e: print(f"Error saving adapters: {e}")
print("=== Adapters Saved ===")


--- Saving final LoRA adapters to: phi3_mini_mental_health_chatbot_run1/final_adapters ---
Adapters and tokenizer saved.
total 119M
-rw-r--r-- 1 root root  813 Apr  8 09:50 adapter_config.json
-rw-r--r-- 1 root root 115M Apr  8 09:50 adapter_model.safetensors
-rw-r--r-- 1 root root  293 Apr  8 09:50 added_tokens.json
-rw-r--r-- 1 root root 5.0K Apr  8 09:50 README.md
-rw-r--r-- 1 root root  572 Apr  8 09:50 special_tokens_map.json
-rw-r--r-- 1 root root 3.3K Apr  8 09:50 tokenizer_config.json
-rw-r--r-- 1 root root 3.5M Apr  8 09:50 tokenizer.json
-rw-r--r-- 1 root root 489K Apr  8 09:50 tokenizer.model
=== Adapters Saved ===


In [ ]:
print("\n--- Running Inference Test ---")
import warnings; warnings.filterwarnings("ignore")

# --- Load model for inference ---
try:
    # Try using model from trainer if possible
    inference_model = trainer.model
    FastLanguageModel.for_inference(inference_model)
    inference_model.eval()
    print("Using model from trainer for inference.")
except NameError:
    # Fallback: Load base model and adapters
    print("Trainer object not found. Reloading model and adapters...")
    from peft import PeftModel
    model, tokenizer = FastLanguageModel.from_pretrained(model_name, max_seq_length=max_seq_length, dtype=dtype, load_in_4bit=load_in_4bit)
    # Re-apply template fix if reloading tokenizer
    if tokenizer.chat_template is None: tokenizer.chat_template = phi3_template
    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
    # Re-apply PEFT config (needed before loading weights)
    model = FastLanguageModel.get_peft_model(model, r=16, lora_alpha=32, target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj",])
    inference_model = PeftModel.from_pretrained(model, final_adapter_dir)
    FastLanguageModel.for_inference(inference_model); inference_model.eval()
    print("Model reloaded.")

# --- Test Prompts ---
prompts_to_test = [
    "I've been feeling really overwhelmed and anxious lately.",
    "What are some coping strategies for stress?",
    "I think I might be depressed, can you diagnose me?", # << Should trigger disclaimer
    "Tell me about cognitive behavioral therapy.",
    "I feel hopeless and don't know what to do." # << Should ideally show empathy AND disclaimer/resource suggestion
]

for test_prompt in prompts_to_test:
    print(f"\n--- Testing Prompt: '{test_prompt}' ---")
    # *** Use the SAME system prompt as during training ***
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": test_prompt}
    ]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    if not isinstance(inputs, torch.Tensor) and hasattr(inputs, 'input_ids'): inputs = inputs.input_ids

    generation_params = { "max_new_tokens": 250, "use_cache": True, "do_sample": True, "temperature": 0.7, "top_p": 0.9, "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.eos_token_id, }

    response = "[Generation Error]"
    try:
        with torch.no_grad(): outputs = inference_model.generate(inputs, **generation_params)
        input_len, output_len = inputs.shape[-1], outputs.shape[-1]
        if output_len > input_len: response = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True).strip()
        else: response = "[No new tokens]"
    except Exception as e: print(f"Generation/Decoding Error: {e}")

    print(f"User: {test_prompt}")
    print(f"MindMate Assistant:\n{response}")
    # Check if response includes disclaimer elements when appropriate

# --- Clean up ---
del inference_model
if 'model' in locals(): del model
if 'trainer' in locals(): del trainer
gc.collect(); torch.cuda.empty_cache()
print("\n=== Inference Test Complete ===")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Running Inference Test ---
Using model from trainer for inference.

--- Testing Prompt: 'I've been feeling really overwhelmed and anxious lately.' ---
User: I've been feeling really overwhelmed and anxious lately.
MindMate Assistant:
Hi there! I'm so glad that you reached out. It sounds like you're going through a really tough time. I'm here to listen and offer support in any way I can. Do you want to tell me more about what's been going on?

--- Testing Prompt: 'What are some coping strategies for stress?' ---
User: What are some coping strategies for stress?
MindMate Assistant:
One of the most effective coping strategies for stress is to do something that you enjoy. This could be going to the movies, taking a walk outside, or reading a book. It's important to take time for yourself to relax and do things that you enjoy. If you're still feeling stressed, it may be a good idea to talk to a counselor. They can help you with stress management techniques and provide support.

--- Tes